In [ ]:
#  AI Travel Planner Agent

## An intelligent travel planning assistant built with LangGraph and Python. 
# This agent collects user input and dynamically plans a trip by checking weather forecasts 
# and suggesting activities based on conditions — all in a conversational and interactive flow.

##  Features

# - Collects source, destination, and travel dates from the user
# - Simulates weather forecast retrieval
# - Suggests activities based on weather conditions
# - Creates a summarized travel itinerary
# - Built using `LangGraph`, a framework for building agent-like workflows

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from random import choice

In [3]:
# ---------- STATE ----------
class PlannerState(TypedDict):
    source: Optional[str]
    destination: Optional[str]
    dates: Optional[str]
    weather: Optional[str]
    activities: Optional[list]
    finalized_plan: Optional[str]
    step: Optional[str]


In [4]:
# ---------- MOCK TOOLS ----------
def get_weather(destination, dates):
    return f"Sunny in {destination} during {dates}"

def suggest_activities(destination, weather):
    if "Sunny" in weather:
        return ["Hiking", "Sightseeing", "Beach"]
    return ["Museum", "Indoor Games"]


In [5]:
# ---------- NODES ----------
def get_user_input(state: PlannerState):
    print("\n Let's plan your trip!")
    state['source'] = input("Enter your current city: ")
    state['destination'] = input("Enter your travel destination: ")
    state['dates'] = input("Enter your travel dates (e.g., Sep 15–20): ")
    state['step'] = "weather"
    return state

In [6]:
def fetch_weather(state: PlannerState):
    print("Checking the weather at your destination...")
    state['weather'] = get_weather(state['destination'], state['dates'])
    state['step'] = "activities"
    return state

In [7]:
def recommend_activities(state: PlannerState):
    print(" Finding activities you can enjoy...")
    state['activities'] = suggest_activities(state['destination'], state['weather'])
    state['step'] = "finalize"
    return state

In [8]:

def finalize_plan(state: PlannerState):
    print("Creating your final travel plan...\n")
    state['finalized_plan'] = (
        f"Trip from {state['source']} to {state['destination']} ({state['dates']})\n"
        f"Weather Forecast: {state['weather']}\n"
        f"Recommended Activities: {', '.join(state['activities'])}"
    )
    state['step'] = "done"
    return state

In [9]:
# ---------- FLOW CONTROL ----------
def flow_decision(state: PlannerState):
    if state['step'] == "weather":
        return "get_weather"
    elif state['step'] == "activities":
        return "recommend_activities"
    elif state['step'] == "finalize":
        return "finalize"
    else:
        return END

In [10]:
# ---------- GRAPH DEFINITION ----------
planner_graph = StateGraph(PlannerState)

planner_graph.add_node("get_input", get_user_input)
planner_graph.add_node("get_weather", fetch_weather)
planner_graph.add_node("recommend_activities", recommend_activities)
planner_graph.add_node("finalize", finalize_plan)

planner_graph.set_entry_point("get_input")

planner_graph.add_conditional_edges("get_input", flow_decision)
planner_graph.add_conditional_edges("get_weather", flow_decision)
planner_graph.add_conditional_edges("recommend_activities", flow_decision)
planner_graph.add_conditional_edges("finalize", flow_decision)

graph = planner_graph.compile()

In [11]:
# ---------- EXECUTE ----------
if __name__ == "__main__":
    initial_state = PlannerState(
        source=None,
        destination=None,
        dates=None,
        weather=None,
        activities=None,
        finalized_plan=None,
        step=None
    )

    print("\n STARTING THE AI TRAVEL PLANNER AGENT...\n")
    final_output = graph.invoke(initial_state)
    print("\n Your Final Travel Plan:\n", final_output['finalized_plan'])



 Starting the AI Travel Planner Agent...


 Let's plan your trip!
Enter your current city: thiruvannamalai
Enter your travel destination: rajasthan
Enter your travel dates (e.g., Sep 15–20): aug 3-10
Checking the weather at your destination...
 Finding activities you can enjoy...
Creating your final travel plan...


 Your Final Travel Plan:
 Trip from thiruvannamalai to rajasthan (aug 3-10)
Weather Forecast: Sunny in rajasthan during aug 3-10
Recommended Activities: Hiking, Sightseeing, Beach
